In [ ]:
import pandas as pd
import numpy as np
import warnings
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib import font_manager, rc
import scipy.cluster.hierarchy as spc
font_name = font_manager.FontProperties(fname="/usr/share/fonts/nanum/nanumGothic.ttf").get_name()
plt.rc('font', family=font_name)
import warnings
warnings.filterwarnings(action='ignore') 
pd.options.display.float_format = '{:.2f}'.format

In [ ]:
from IPython.display import display_html
def display_side_by_side(*args):
    html_str=''
    for df in args:
        html_str+=df.to_html()
    display_html(html_str.replace('table','table style="display:inline"'),raw=True)

In [ ]:
#data = pd.read_excel('data/alram_ds.xls',sheet_name=None, encoding='UTF-8')
#df_list = [ v for k,v in data.items()] 
#data = pd.concat(df_list ,axis=0)

## 1. 알람 데이터 로드 및 전처리

In [ ]:
data = pd.read_csv('data/alaram.csv', encoding='UTF-8',index_col=0)

In [ ]:
#행렬 확인
data.shape

In [ ]:
#불필요 컬럼 제거
cols = ['처리.현황', '담당자', '처리.이력', '확인.시간', '유지보수.상태']
data.drop(columns=cols,axis=1,inplace=True)

In [ ]:
#컬럼 이름 수정
data.columns = ['degree', 'alarm_name','time','clear_time','last_time','group_path','system_name','hostname','ip','resource_name','full_path','condition']

In [ ]:
#추가 컬럼 생성
data['time'] = pd.to_datetime(data['time'], format='%Y-%m-%d %H:%M:%S')
data['clear_time'] = pd.to_datetime(data['clear_time'], format='%Y-%m-%d %H:%M:%S')
data['last_time'] = data['clear_time'] - data['time']
data['last_time'] = data['last_time'].dt.seconds
data['last_time'] = data['last_time'].fillna(0)
data['time_month'] = data['time'].dt.month
data['time_day'] = data['time'].dt.day
data['time_hour'] = data['time'].dt.hour
data['time_date']= data['time'].dt.strftime('%Y-%m-%d')
data['time_date_hour']= data['time'].dt.strftime('%Y-%m-%d %H')
full_group = data['group_path'].str.split('>', expand=True)
data['group1'] = full_group[0]
data['group2'] = full_group[1]
data['group3'] = full_group[2]
data['group4'] = full_group[3]
ipaddr = data['ip'].str.split('.', expand=True)
ipaddr[3] = "0"
data['C_class']= ipaddr[0] + '.' + ipaddr[1] + '.' + ipaddr[2] + '.' + ipaddr[3]
ipaddr[3] = "0"
ipaddr[2] = "0"
data['B_class']= ipaddr[0] + '.' + ipaddr[1] + '.' + ipaddr[2] + '.' + ipaddr[3]

In [ ]:
#데이터 확인
data.head()

## 2. 데이터 분석

### 노드 별 심각도 건수

In [ ]:
df = data.groupby(['system_name','degree'])['alarm_name'].count().reindex().reset_index()
alram_count = df.pivot_table(values='alarm_name', index=df.system_name, columns='degree', aggfunc='first',fill_value=0).reset_index()
alram_count['total_count'] = alram_count['경고'] + alram_count['주의'] + alram_count['심각']
alram_count.sort_values('total_count',ascending=False).head()

### 등급별 건수

In [ ]:
df = data.groupby(['degree'])['alarm_name'].count().reindex().reset_index()
df['per'] = (df.alarm_name/df.alarm_name.sum())*100
df['per']= df['per'].round(2)
df.columns = ['degree', 'count' , 'per']
fig = plt.figure(figsize=(15, 7))
ax1 = fig.add_subplot(1, 2, 1)
ax2 = fig.add_subplot(1, 2, 2)
ax2.axis('off')
ax2.axis('tight')
ax1.pie(df.per, explode=[0.05,0.05,0.05],labels=df.degree, autopct='%1.1f%%',colors=['orange','red','yellow'],textprops={'fontsize': 20},
       shadow=True, startangle=90)
ax2.table(cellText=df.values, colLabels=df.columns, loc='center')
df

### 일별 등급 알람건수

In [ ]:
df = data.groupby(['time_date','degree'])['alarm_name'].count().reindex().reset_index()
df = df[df['degree'] == '심각']
df.sort_values('alarm_name',ascending=False).head()

In [ ]:
df = data.groupby(['time_date','degree'])['alarm_name'].count().reindex().reset_index()
df = df[df['degree'] == '주의']
df.sort_values('alarm_name',ascending=False).head()

In [ ]:
df = data.groupby(['time_date','degree'])['alarm_name'].count().reindex().reset_index()
df = df[df['degree'] == '경고']
df.sort_values('alarm_name',ascending=False).head()

### 노드별 발생 알람이름 건수

In [ ]:
df = data.groupby(['system_name','alarm_name'])['time'].count().reindex().reset_index()
df = df.pivot_table(values='time', index=df.system_name, columns='alarm_name', aggfunc='first',fill_value=0).reset_index()
df = df.set_index('system_name')
#df.reindex(columns='시스템명')
df['total_count']= df.sum(axis=1)
df.sort_values('total_count',ascending=False).head()

In [ ]:
df = data.groupby(['system_name'])['time'].count().reindex().reset_index().sort_values('time')
df[df['time'] == 1]

### 

In [ ]:
rotation = 45

colors = ['red','orange','yellow']
f,ax=plt.subplots(1,1,figsize=(18,8))
sns.countplot('group1',hue='degree',data=data,order=data.group1.value_counts().iloc[:10].index, palette=sns.xkcd_palette(colors))
ax.set_title('group 별 심각도')
plt.xticks(rotation=rotation)
plt.show()
df = data.groupby(['group1','degree'])['time'].count().reindex().reset_index().sort_values('time',ascending=False)
df.columns = ['group1','degree','count']
df_1 = df[df['degree'] == '심각'].head(10)
df_2 = df[df['degree'] == '경고'].head(10)
df_3 = df[df['degree'] == '주의'].head(10)
display_side_by_side(df_1,df_2,df_3)

colors = ['red','orange','yellow']
f,ax=plt.subplots(1,1,figsize=(18,8))
sns.countplot('group2',hue='degree',data=data,order=data.group2.value_counts().iloc[:10].index, palette=sns.xkcd_palette(colors))
ax.set_title('group2 심각도')
plt.xticks(rotation=rotation)
plt.show()
df = data.groupby(['group2','degree'])['time'].count().reindex().reset_index().sort_values('time',ascending=False)
df.columns = ['group2','degree','count']
df_1 = df[df['degree'] == '심각'].head(10)
df_2 = df[df['degree'] == '경고'].head(10)
df_3 = df[df['degree'] == '주의'].head(10)
display_side_by_side(df_1,df_2,df_3)

colors = ['red','orange','yellow']
f,ax=plt.subplots(1,1,figsize=(18,8))
sns.countplot('group3',hue='degree',data=data,order=data.group3.value_counts().iloc[:10].index, palette=sns.xkcd_palette(colors))
ax.set_title('group3 심각도')
plt.xticks(rotation=rotation)
plt.tick_params(axis='both', which='minor', labelsize=0.2)
plt.show()
df = data.groupby(['group3','degree'])['time'].count().reindex().reset_index().sort_values('time',ascending=False)
df.columns = ['group3','degree','count']
df_1 = df[df['degree'] == '심각'].head(10)
df_2 = df[df['degree'] == '경고'].head(10)
df_3 = df[df['degree'] == '주의'].head(10)
display_side_by_side(df_1,df_2,df_3)

colors = ['red','orange','yellow']
f,ax=plt.subplots(1,1,figsize=(18,8))
sns.countplot('system_name',hue='degree',data=data,order=data.system_name.value_counts().iloc[:10].index, palette=sns.xkcd_palette(colors))
ax.set_title('시스템별 심각도')
plt.xticks(rotation=rotation)
plt.tick_params(axis='both', which='minor', labelsize=0.2)
plt.show()
df = data.groupby(['system_name','degree'])['time'].count().reindex().reset_index().sort_values('time',ascending=False)
df.columns = ['system_name','degree','count']
df_1 = df[df['degree'] == '심각'].head(10)
df_2 = df[df['degree'] == '경고'].head(10)
df_3 = df[df['degree'] == '주의'].head(10)
display_side_by_side(df_1,df_2,df_3)

colors = ['orange','yellow','red']
f,ax=plt.subplots(1,1,figsize=(18,8))
sns.countplot('alarm_name',hue='degree',data=data,order=data.alarm_name.value_counts().iloc[:10].index, palette=sns.xkcd_palette(colors))
ax.set_title('알람 이름 별 심각도')
plt.xticks(rotation=rotation)
plt.tick_params(axis='both', which='minor', labelsize=0.2)
plt.show()
df = data.groupby(['alarm_name','degree'])['time'].count().reindex().reset_index().sort_values('time',ascending=False)
df.columns = ['alarm_name','degree','count']
df_1 = df[df['degree'] == '심각'].head(10)
df_2 = df[df['degree'] == '경고'].head(10)
df_3 = df[df['degree'] == '주의'].head(10)
display_side_by_side(df_1,df_2,df_3)


colors = ['red','orange','yellow']
f,ax=plt.subplots(1,1,figsize=(18,8))
sns.countplot('C_class',hue='degree',data=data,order=data.C_class.value_counts().iloc[:10].index, palette=sns.xkcd_palette(colors))
ax.set_title('C_class')
plt.xticks(rotation=rotation)
plt.tick_params(axis='both', which='minor', labelsize=0.2)
plt.show()
df = data.groupby(['C_class','degree'])['time'].count().reindex().reset_index().sort_values('time',ascending=False)
df.columns = ['C_class','degree','count']
df_1 = df[df['degree'] == '심각'].head(10)
df_2 = df[df['degree'] == '경고'].head(10)
df_3 = df[df['degree'] == '주의'].head(10)
display_side_by_side(df_1,df_2,df_3)


colors = ['red','orange','yellow']
f,ax=plt.subplots(1,1,figsize=(18,8))
sns.countplot('B_class',hue='degree',data=data,order=data.B_class.value_counts().iloc[:10].index, palette=sns.xkcd_palette(colors))
ax.set_title('B_class')
plt.xticks(rotation=rotation)
plt.tick_params(axis='both', which='minor', labelsize=0.2)
plt.show()
df = data.groupby(['B_class','degree'])['time'].count().reindex().reset_index().sort_values('time',ascending=False)
df.columns = ['B_class','degree','count']
df_1 = df[df['degree'] == '심각'].head(10)
df_2 = df[df['degree'] == '경고'].head(10)
df_3 = df[df['degree'] == '주의'].head(10)
display_side_by_side(df_1,df_2,df_3)

### 3. time_date(특정일) time_hour(시간) time_weekday(요일) time_day(일) time_week(주) 별 알람 건수

In [ ]:
df = data.groupby(['system_name','time_date'])['time'].count().reindex().reset_index()
df.columns =['system_name', 'time_date', 'total_count']
df = df.pivot_table(values='total_count', index=df['time_date'], columns=df['system_name'], aggfunc='first',fill_value=0)
df['total_count'] = df.sum(axis=1)
df = df.sort_values('total_count',ascending=False)

df_hour = data.groupby(['time_hour','degree'])['time'].count().reindex().reset_index()
df_hour.columns = ['time_hour', 'degree' , 'total_count']

data['time_weekday'] = data['time'].dt.weekday
df_weekday = data.groupby(['time_weekday','degree'])['time'].count().reindex().reset_index()
df_weekday.columns = ['time_weekday', 'degree' , 'total_count']

df_days = data.groupby(['time_day','degree'])['time'].count().reindex().reset_index()
df_days.columns = ['time_day', 'degree' , 'total_count']

data['time_week'] = data['time'].dt.week
df_week = data.groupby(['time_week','degree'])['time'].count().reindex().reset_index()
df_week.columns = ['time_week', 'degree' , 'total_count']

In [ ]:
df = data.groupby(['time_date','degree'])['time'].count().reindex().reset_index()
df.columns = ['time_date', 'degree' , 'total_count']
display_side_by_side(df[df.degree.str.match('심각')==True].sort_values('total_count', ascending=False).head(),
        df[df.degree.str.match('주의')==True].sort_values('total_count', ascending=False).head(),
        df[df.degree.str.match('경고')==True].sort_values('total_count', ascending=False).head())

In [ ]:
display_side_by_side(df_hour[df_hour.degree.str.match('심각')==True].sort_values('total_count', ascending=False).head(),
df_hour[df_hour.degree.str.match('주의')==True].sort_values('total_count', ascending=False).head(),
df_hour[df_hour.degree.str.match('경고')==True].sort_values('total_count', ascending=False).head())

In [ ]:
display_side_by_side(df_days[df_days.degree.str.match('심각')==True].sort_values('total_count', ascending=False).head(),
df_days[df_days.degree.str.match('주의')==True].sort_values('total_count', ascending=False).head(),
df_days[df_days.degree.str.match('경고')==True].sort_values('total_count', ascending=False).head())

In [ ]:
display_side_by_side(df_weekday[df_weekday.degree.str.match('심각')==True].sort_values('total_count', ascending=False).head(),
df_weekday[df_weekday.degree.str.match('주의')==True].sort_values('total_count', ascending=False).head(),
df_weekday[df_weekday.degree.str.match('경고')==True].sort_values('total_count', ascending=False).head())


In [ ]:
display_side_by_side(df_week[df_week.degree.str.match('심각')==True].sort_values('total_count', ascending=False).head(),
df_week[df_week.degree.str.match('주의')==True].sort_values('total_count', ascending=False).head(),
df_week[df_week.degree.str.match('경고')==True].sort_values('total_count', ascending=False).head())


In [ ]:
sns.factorplot(data=df, x="time_date", y="total_count",hue='degree',legend_out=True, size=3, aspect=4)
plt.xticks(rotation='vertical')
plt.title('일자별 알람 추이')

sns.factorplot(data=df_hour, x="time_hour", y="total_count",hue='degree',legend_out=True, size=3, aspect=4)
plt.title('시간별 알람 추이')

sns.factorplot(data=df_days, x="time_day", y="total_count",hue='degree',legend_out=True, size=3, aspect=4)
plt.title('일자별 알람 추이(1일~31일)')
sns.factorplot(data=df_weekday, x="time_weekday", y="total_count",hue='degree',legend_out=True, size=3, aspect=4)
plt.title('요일별 알람 추이')
sns.factorplot(data=df_week, x="time_week", y="total_count",hue='degree',legend_out=True, size=3, aspect=4)
plt.title('주별 알람 추이')

### 4. 특정 일에 발생한 알람 확인을 위한 코드

In [ ]:
df = data.groupby(['time_date','degree','alarm_name','system_name','condition'])['ip'].count().reindex().reset_index()
df.columns = ['time_date', 'degree' ,'alarm_name','system_name','condition', 'total_count']

display(df[df['degree'] == '심각'].sort_values('total_count', ascending=False).head(30))
display(df[df['degree'] == '경고'].sort_values('total_count', ascending=False).head(30))
display(df[df['degree'] == '주의'].sort_values('total_count', ascending=False).head(30))

df = data.groupby(['time_date_hour','degree'])['ip'].count().reindex().reset_index()
df.columns = ['time_date_hour', 'degree' , 'total_count']

sns.factorplot(data=df, x="time_date_hour", y="total_count",hue='degree',legend_out=True, size=20, aspect=4 ,)
plt.xticks(rotation='vertical')
plt.title('시간별 알람 추이')

display(df[df['degree'] == '심각'].sort_values('total_count', ascending=False).head())
index_time = df[df['degree'] == '심각'].sort_values('total_count', ascending=False).head().time_date
for time in index_time:
   df[(df['degree'] == '심각')&(df['time'].astype('str').isin(time))]).sort_values('total_count', ascending=False).head(10))

((df['degree'] == '심각')&(df['time'].astype('str').isin(time)))
#display(df[df['degree'] == '경고'].sort_values('total_count', ascending=False).head())
#display(df[df['degree'] == '주의'].sort_values('total_count', ascending=False).head())

In [ ]:
df = data.groupby(['time','degree','alarm_name','system_name','condition'])['ip'].count().reindex().reset_index()
df.columns = ['time', 'degree' ,'alarm_name','system_name','condition', 'total_count']

display_side_by_side(df[df['degree'] == '심각'].sort_values('total_count', ascending=False).head(10))
display_side_by_side(df[df['degree'] == '경고'].sort_values('total_count', ascending=False).head(10))
display_side_by_side(df[df['degree'] == '주의'].sort_values('total_count', ascending=False).head(10))

### 5. 발생 알람 건수 분포 히스토그램

In [ ]:
df = data.groupby(['system_name'])['alarm_name'].count().reindex().reset_index()
sns.distplot( df["alarm_name"], bins=20 )

In [ ]:
df.sort_values('alarm_name',ascending=False)

### 6. 노드 단위로 발생 알람 건수를 groupby 해서 75% (3분위 수) 120건 미만으로 발생한 노드는 제외함 (Cluster Step 1)

display(df[(df['degree'] == '심각')&((df['time'] >= '2018-07-21')&(df['time'] <= '2018-07-22'))].sort_values('total_count', ascending=False).head(100))


In [ ]:
data_df = data.groupby(['system_name'])['time'].count().reindex().reset_index()

In [ ]:
data_df = data_df.rename(columns={'time': 'count'})

In [ ]:
data_df.describe() 

In [ ]:
data_df = data_df[data_df['count'] > 120]

In [ ]:
node_names = data_df.sort_values('count',ascending=False).system_name.values

In [ ]:
c_nodes = data[data['system_name'].isin(node_names)==True]

In [ ]:
data_df_date = c_nodes.groupby(['time_date','system_name'])['time'].count().reindex().reset_index()
data_df_date = data_df_date.pivot_table(values='time',columns='system_name',index=data_df_date['time_date'],aggfunc='first',fill_value=0)

### 7. 노드별로 1달 동안 일별로 발생한 알람 건수를 가지고 각 노드들 끼리 알람 건수에 대한 상관도 측정 (Cluster Step 2)

In [ ]:
data_df_date

for node in data_df_date.reindex(sorted(data_df_date.columns), axis=1).columns:
    print('>>>>>>>> '+node)
    fig = plt.figure(figsize=(15, 7))
    ax1 = fig.add_subplot(2, 1, 1)
    ax2 = fig.add_subplot(2, 1, 2)
    plt.subplots_adjust(top=0.5)
    ax2.axis('off')
    ax2.axis('tight')
    c_nodes = data[data['system_name']==node]
    dataset = c_nodes.groupby(['time_date','alarm_name'])['time'].count().reindex().reset_index().sort_values('time', ascending=False).head(10)
    ax2.table(cellText=dataset.values, colLabels=dataset.columns, loc='bottom')
    plt.setp(ax1.get_xticklabels(), rotation=90)
    sns.pointplot(x=data_df_date.index, y=node, data=data_df_date,ax=ax1,rotation=90)
    plt.show()
    plt.clf()

In [ ]:
corrmat = data_df_date.corr()

### 8. 상관분석 결과 (Cluster Step 3)

In [ ]:
corrmat

In [ ]:
f, ax = plt.subplots(figsize=(100,100))
sns_plot = sns.heatmap(corrmat, ax=ax, cmap="YlGnBu", linewidths=0.1)
sns_plot.figure.savefig("output.png")

### 9. 상관분석 결과 상관도가 90% 이상인 노드들끼리 클러스터링 (Cluster Result)

In [ ]:
clusters = []
for node in corrmat.columns:
    clusters.append(corrmat[corrmat[node] >= 0.9].index)
clusters_new = []
for cluster in clusters:
    if len(cluster) > 1:
        clusters_new.append(list(cluster))
clusters_new = list(set([tuple(set(node)) for node in clusters_new]))

dictt = {}
for i in range(0,len(clusters_new)):
    dictt.update({'cluster{}'.format(i) : list(clusters_new[i])})

df = pd.concat([pd.Series(clusters_new[0],name='cluster0'),pd.Series(clusters_new[1],name='cluster1')], axis=1)
for i in range(2,len(clusters_new)):
    df = pd.concat([df,pd.Series(clusters_new[i],name='cluster{}'.format(i))], axis=1)
#df.fillna('',inplace=True)

In [ ]:
len(clusters_new)

### 10. 클러스터에 속한 각각의 노드의 한달 알람 발생 추의와 주요 알람 지표를 출력 (Visualization for Cluster Result)

In [ ]:
for cluster in clusters_new:
    print('######### '+ str(cluster))
    for node in cluster:
        print('>>>>>>>> '+node)
        fig = plt.figure(figsize=(15, 7))
        ax1 = fig.add_subplot(2, 1, 1)
        ax2 = fig.add_subplot(2, 1, 2)
        plt.subplots_adjust(top=0.5)
        ax2.axis('off')
        ax2.axis('tight')
        c_nodes = data[data['system_name']==node]
        dataset = c_nodes.groupby(['time_date','alarm_name'])['time'].count().reindex().reset_index().sort_values('time', ascending=False).head(10)
        ax2.table(cellText=dataset.values, colLabels=dataset.columns, loc='bottom')
        plt.setp(ax1.get_xticklabels(), rotation=90)
        sns.pointplot(x=data_df_date.index, y=node, data=data_df_date,ax=ax1,rotation=90)
        plt.show()
        plt.clf()

### 10-1 클러스터 별로 한달 알람 발생 추의와 주요 알람 지표를 출력

In [ ]:
data_df_date = data_df_date.reset_index()

In [ ]:
i = 0
for cluster in clusters_new:
    print('>>>>>>>>>cluster{}'.format(i))
    test_data = pd.melt(data_df_date, id_vars=['time_date'],value_vars=cluster)
    fig = plt.figure(figsize=(15, 7))
    ax = sns.pointplot(x="time_date", y="value",hue=test_data.system_name, data=test_data)
    plt.xticks(rotation='vertical')
    plt.show()
    plt.clf()
    i += 1

In [ ]:
i = 0
for cluster in clusters_new:
    print('>>>>>>>cluster{}\n{}'.format(i,cluster))
    c_nodes = data[data['system_name'].isin(cluster)]
    c_nodes = c_nodes.groupby(['time_date','alarm_name','system_name'])['time'].count().reindex().reset_index().sort_values('time_date', ascending=True)
    c_nodes = c_nodes.groupby(['alarm_name','system_name'])['time'].sum().reindex().reset_index().sort_values('time', ascending=False)
    c_nodes = c_nodes[c_nodes.time > c_nodes.time.mean()]
    display_side_by_side(c_nodes)
    i += 1

data[(data.system_name == 'tsgwap') & (data.time_date=='2018-08-03')]

pd.set_option('display.max_colwidth', -1)
data[(data.system_name == 'tsgwap') & (data.time_date=='2018-08-03')].condition

### 11-1. 클러스터 0번에 속한 노드의 주요 지표 디스크 Top I/O 처리율에 대한 노드별 심각도 알람 건수 추이 출력
> (클러스터 0번이 첫번재 결과라 선택 함) - cluster0 ('HFEU-TREC234', 'HKCC-TREC231', 'HKCC-TREC236')
> 날짜별 현황

In [ ]:
df = data[(data.alarm_name == '디스크 Top I/O 처리율') & (data['system_name'].isin(clusters_new[0]))]
#df = df[df['system_name'] == clusters_new[0][0]]
df = df.groupby(['time_date','degree','system_name'])['ip'].count().reindex().reset_index()
for node in clusters_new[0]:
    df_2 = df[df['system_name'] == node]
    df_2 = df_2.pivot_table(values='ip', index=df.time_date, columns='degree', aggfunc='first',fill_value=0).reset_index()
    display(df_2)
    df_2 = pd.melt(df_2,id_vars=['time_date'],value_vars=['주의','경고','심각'])
    #display(df_2)
    sns.factorplot(data=df_2, x="time_date", y="value",hue='degree',legend_out=True, size=3, aspect=4)
    plt.title(node)
    plt.xticks(rotation='vertical')

> #### 234 서버는 주말 관계없이 알람이 발생하는 하므로 서버 리소스 사용률이 평소에도 높은것으로 추측
> #### 231,236 서버는 주말에 알람이 거의 발생하지 않는 것으로 확인

### 11-2. 클러스터 0번에 속한 노드의 주요 지표 디스크 Top I/O 처리율에 대한 노드별 심각도 알람 건수 추이 출력
> (클러스터 0번이 첫번재 결과라 선택 함) - cluster0 ('HFEU-TREC234', 'HKCC-TREC231', 'HKCC-TREC236')
> 요일별 현황

In [ ]:
df = data[(data.alarm_name == '디스크 Top I/O 처리율') & (data['system_name'].isin(clusters_new[0]))]
#df = df[df['system_name'] == clusters_new[0][0]]
df = df.groupby(['time_weekday','degree','system_name'])['ip'].count().reindex().reset_index()
for node in clusters_new[0]:
    df_2 = df[df['system_name'] == node]
    df_2 = df_2.pivot_table(values='ip', index=df.time_weekday, columns='degree', aggfunc='first',fill_value=0).reset_index()
    df_2 = pd.melt(df_2,id_vars=['time_weekday'],value_vars=['주의','경고','심각'])
    display(df_2)
    sns.factorplot(data=df_2, x="time_weekday", y="value",hue='degree',legend_out=True, size=3, aspect=4)
    plt.title(node)

> ### 234 서버는 다른 요일에 비해 수요일에 알람이 적게 발생하고
> ### 231, 236 서버는 토요일 일요일에 알람이 거의 발생하지 않음

### 11-3. 클러스터 0번에 속한 노드의 주요 지표 디스크 Top I/O 처리율에 대한 노드별 심각도 알람 건수 추이 출력
> (클러스터 0번이 첫번재 결과라 선택 함) - cluster0 ('HFEU-TREC234', 'HKCC-TREC231', 'HKCC-TREC236')
> 알람 발생 시점의 실제 성능 값

In [ ]:
df = data[(data.alarm_name == '디스크 Top I/O 처리율') & (data['system_name'].isin(clusters_new[0]))]
condition  = df.condition.str.split(' ', expand=True)
df['threshold'] = condition[6].astype(float)
df['current'] = condition[3].str.extract('(\d+(?:\.\d+)?)').astype(float)
df = df.groupby(['time_date','system_name','degree'])['current'].mean().reindex().reset_index()
for node in clusters_new[0]:
    print(node)
    df_2 = df[df['system_name'] == node]
    df_2 = df_2.pivot_table(values='current', index=df_2.time_date, columns='degree', aggfunc='first',fill_value=0).reset_index()
    display(df_2)
    df_2 = pd.melt(df_2,id_vars=['time_date'],value_vars=['주의','경고','심각'])
    sns.factorplot(data=df_2, x="time_date", y="value",hue='degree',legend_out=True, size=3, aspect=4)
    plt.title(node)
    plt.xticks(rotation='vertical')

### 일자별로 알람 발생 당시의 성능 값 추이